### Imports

In [103]:
import numpy as np
import pandas as pd

### Read data from logs
Each of the provided logs contains frequencies for each of the 1280 tero-instances, for 100 pufs (100 different board)

In [104]:
def extract_frequencies(csv_file, counter_bits):
    data = pd.read_csv(csv_file, sep=',', skiprows=1, names=["BoardNumber","DataSent", "#Sent", "DataReceived", "#Received"])
    responsesRaw = np.array(data['DataReceived'])
    responses = []
    for responseRaw in responsesRaw:
        response = []
        for i in range(0, len(responseRaw), int(counter_bits/4)):
            response.append(int(responseRaw[i:i+int(counter_bits/4)], 16))
        responses.append(response)
    responses = np.array(responses)
    return np.transpose(responses) # for each row, an instance of tero. for each column, the value in that fpga

In [105]:
freqs1 = extract_frequencies("TERO/log_20161008U235656_Small_ChosenRepetition_ChosenTime_22.0C.csv", 32)
freqs2 = extract_frequencies("TERO/log_20161009U002207_Small_ChosenRepetition_ChosenTime_22.0C.csv", 32)
freqs3 = extract_frequencies("TERO/log_20161009U023930_Small_ChosenRepetition_ChosenTime_-0.0C.csv", 32)
freqs4 = extract_frequencies("TERO/log_20161009U110717_Small_ChosenRepetition_ChosenTime_44.1C.csv", 32)

In [106]:
print(freqs1.shape)
print(freqs1)

(1280, 100)
[[  71870  239361  165805 ...  126003   96193   72745]
 [  61075 1131202  936723 ...   63300  122902  123455]
 [  68815   93162  373604 ...   43073   73202  447935]
 ...
 [  85216  104602   91431 ...  142617  219258  164340]
 [ 163790   66403  118687 ...  133235  124278  148629]
 [ 143384  118096   52582 ...  194886   83166  447925]]


### Group frequencies in batches

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16

[0] 0 4 8 12 16
[1] 1 5 9 13 17

[0] 0 8 16  x 160
[1] 1 9 17
[2] 2 10 18
[3] 3 ...
[4] 4 12 20
...
[7]

In [107]:
def group_frequencies(freqs):
    # Based on the assumption we have 8 different cell types, we group each type in the right column
    NUM_INSTANCES = freqs.shape[0]  # 1280
    NUM_FPGA = freqs.shape[1]   # 100
    NUM_TYPES = 8
    BATCH_SIZE = 16
    batchArray = np.zeros((int(NUM_INSTANCES/NUM_TYPES),NUM_TYPES,NUM_FPGA))
    for fpga in range(0, NUM_FPGA):
        for instance_type in range(0, NUM_TYPES):
            batchArray[:,instance_type,fpga] = freqs[instance_type::NUM_TYPES,fpga] # from the starting index, we sample every 8 elements
    # batcharray: 160x8x100
    BATCH_COLUMNS = int(NUM_INSTANCES/BATCH_SIZE)   # 80
    batchResponse = np.zeros((BATCH_SIZE,BATCH_COLUMNS,NUM_FPGA))
    for fpga in range(0, NUM_FPGA):
        fpgaBatch = np.zeros((BATCH_SIZE, BATCH_COLUMNS))
        col = 0
        for instance_type in range(0,NUM_TYPES):
            # get old column
            current_col = batchArray[:,instance_type,fpga]
            # create 10 new columns
            for index in range(0, len(current_col), BATCH_SIZE):
                fpgaBatch[:,col] = current_col[index:index+BATCH_SIZE]
                col += 1
            # continue for each of the 8 columns (8x10 = 80 columns)
        # set the new batch
        batchResponse[:,:,fpga] = fpgaBatch[:,:]
    return batchResponse

In [108]:
batched1 = group_frequencies(freqs1)
batched2 = group_frequencies(freqs2)
batched3 = group_frequencies(freqs3)
batched4 = group_frequencies(freqs4)

In [109]:
batched1[:,0,0] # first batch, first fpga
batched1.shape

(16, 80, 100)

### 2Comp Algorithm
Choosing 2 frequencies for each batch, the response is given by comparing them:

* Choose 0 < i < BATCH_SIZE
* Choose 0 < j < BATCH_SIZE, i ~= j
* If f(i) > f(j) => res = 1, else => res = 0

In this way 1 bit for each batch is extracted (in our case 80 bits)


In [110]:
def alg_2comp_chosen_i_j (i,j,puf_data):

    NUM_BATCHES = puf_data.shape[1]
    NUM_FPGA = puf_data.shape[2]

    response = np.zeros((NUM_BATCHES,NUM_FPGA))
    for currFpga in range (1,NUM_FPGA): #num of fpga
        for currBatch in range (1,NUM_BATCHES): #num of batches
            if i == j :
                response[currBatch,currFpga] = 0
            else:
                response[currBatch,currFpga] = 1 if puf_data[i,currBatch,currFpga] > puf_data[j,currBatch,currFpga] else 0
    
    return response # (80 bit, 100 fpga)

In [111]:
alg_2comp_chosen_i_j (1,2,batched1).shape


(80, 100)

In [112]:
def get_res_array (experiment):
    """ This function takes an experiment (data for a certain temp) and returns all possible 80bits responses (for each i,j combination)"""
    BATCH_SIZE = 16
    SINGLE_RESPONSE_BITS = 80
    NUM_FPGA = 100
    
    res_array = np.zeros((SINGLE_RESPONSE_BITS,BATCH_SIZE*(BATCH_SIZE-1), NUM_FPGA)) #80,256,100
    #res_array = np.zeros((BATCH_SIZE*BATCH_SIZE,SINGLE_RESPONSE_BITS, NUM_FPGA)) #256,80,100
    curr_ij_couple = 0

    for i_index in range(0, BATCH_SIZE):
        for j_index in range (0,BATCH_SIZE):
            if i_index != j_index :
                res_array[:,curr_ij_couple,:] = alg_2comp_chosen_i_j(i_index,j_index,experiment)
                curr_ij_couple += 1
    
    return res_array


In [113]:
data = np.array([
    get_res_array(batched1),
    get_res_array(batched2),
    get_res_array(batched3),
    get_res_array(batched4)
], dtype=np.int)

In [114]:
data.shape

(4, 80, 256, 100)

In [97]:
print(data[0,:,0,1])
data[0,:,:,1].shape

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]


(80, 256)

### Analysis

#### Uniformity
The uniformity metric (also called randomness by Yu et al. in [53])
determines how uniform the proportion of 0′ 𝑠 and 1′ 𝑠 are in the PUF
response

In [98]:
def calc_uniformity(response):
    return np.sum([response[i] for i in range(0, len(response))]) / len(response)

def worst_uniformity(data):
    max = 0.5   # ideal value
    for puf_num in range(0, data.shape[3]):
        for exp_num in range(0, data.shape[0]):
            for batch_num in range(0, data.shape[2]):
                u = calc_uniformity(data[exp_num,:,batch_num,puf_num])
                if np.abs(u - 0.5) > np.abs(max - 0.5):
                    max = u
    return max

In [102]:
worst_uniformity(data)

0.0

#### Reliability
It determines how efficiently a PUF can generate the same response
at different operating conditions (ambient temperatures or supply volt-
ages) over a period of time for a given challenge

In [100]:
def hamming_distance(arr1, arr2): 
    distance = 0
    L = len(arr1)
    for i in range(L):
        if arr1[i] != arr2[i]:
            distance += 1
    return distance

def hd_intra(puf_responses):
    return np.sum([
        hamming_distance(puf_responses[0,:], puf_responses[exp_num,:]) / len(puf_responses[0,:])
        for exp_num in range(1, len(puf_responses))]) / len(puf_responses[0,:])
    
def reliability(puf_responses):
    return 1 - hd_intra(puf_responses)

def worst_reliability(data):
    min = 1 # ideal value
    for puf_num in range(0, data.shape[3]):
        for batch_num in range(0, data.shape[2]):
            r = reliability(data[:,:,batch_num,puf_num])
            if r < min:
                min = r
    return min

In [101]:
worst_reliability(data)

0.995625

In [30]:
arr_try1 = np.array([[1,2,3],[4,5,6]])
arr_try2 = np.array([[7,8,9],[10,11,12]])
arr_try3 = np.array([[13,14,15],[16,17,18]])

arr_3d = np.array([arr_try1, arr_try2,arr_try3])

arr_3d[1,:]

array([[ 7,  8,  9],
       [10, 11, 12]])

#### Uniqueness
It measures the variation of responses obtained from different chips
for the same set of challenges

In [31]:
def calc_uniqueness(data):
    num_bits = data.shape[0]
    num_devices = data.shape[1]
    return 2/(num_devices*(num_devices-1)) * np.sum([
        np.sum([
            hamming_distance(data[:,i], data[:,j])/num_bits
            for j in range(i, num_devices)])  
        for i in range(0, num_devices-2)])

def worst_uniqueness(data):
    n_esperiments = data.shape[0]
    max = 0.5   # ideal value
    for exp_num in range(0, n_esperiments):
        for batch_num in range(0, data.shape[2]):
            u = calc_uniqueness(data[exp_num,:,batch_num,:])
            if np.abs(u - 0.5) > np.abs(max - 0.5):
                max = u
    return max

In [32]:
worst_uniqueness(data)

KeyboardInterrupt: 

### Some samples

In [ ]:
def bit_array_to_string(arr):
    response_char = ''
    for i in range(0, len(arr)):
        response_char = response_char + ('1' if arr[i] else '0')
    return response_char  

def print_ids_per_fpga(data):
    num_batches = data.shape[1] #80
    out = []
    for curr_batch in range(0, num_batches):
        out.append(bit_array_to_string(data[:,curr_batch]))
    return out

In [ ]:
# Let's print all the first 10 ids of the first board at 20°C
print_ids_per_fpga(data[0,:,:,0])[0:10]

['1101111000010010100110100010100000001101011110110',
 '0110000000001111001011011101101011101110011010011',
 '1000101001010010100110111101101001111110111101100',
 '1001101100001010001110100010001011110001111001010',
 '0001110000110010100010100001001011101001001010010',
 '0000000100000001100010101011001100100001011001011',
 '1111001100010010101100100010000100111011111010010',
 '0101010000110010100110001001100111001010110000110',
 '1001111101001011100011011000100010101111101001010',
 '1010011100001110101000111011101011100000011111100']